In [14]:
import os
import tarfile
import boto3
from transformers import AutoModelForSequenceClassification, AutoTokenizer,pipeline
from huggingface_hub import HfApi, HfFolder, create_repo

2025-09-12 19:04:25.247071: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
bucket = "sagemaker-eu-north-1-013001369529"
job_name = "huggingface-pytorch-training-2025-09-12-02-55-33-638"
s3_key="s3://sagemaker-eu-north-1-013001369529/huggingface-pytorch-training-2025-09-12-02-55-33-638/output/model.tar.gz"
local_tar_path = "toxic_model.tar.gz"
extract_dir = "model_artifacts"
repo_id = "Youssef-El-SaYed/toxic-comment-classifier"  

In [4]:
s3 = boto3.client("s3")
# Extract bucket name and key correctly
s3_bucket = bucket
s3_object_key = s3_key.replace(f"s3://{bucket}/", "")

# Download model.tar.gz
s3.download_file(s3_bucket, s3_object_key, local_tar_path)
print("Downloaded:", local_tar_path)

Downloaded: toxic_model.tar.gz


In [5]:
os.makedirs(extract_dir, exist_ok=True)

with tarfile.open(local_tar_path, "r:gz") as tar:
    tar.extractall(path=extract_dir)

print("Extracted to:", extract_dir)


/tmp/ipykernel_19973/3292829401.py:4: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_dir)


Extracted to: model_artifacts


In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
repo_name = "toxic-comment-classifier"
repo_id = f"Youssef-El-SaYed/{repo_name}"


In [12]:
token = HfFolder.get_token()

create_repo(repo_id, repo_type="model", exist_ok=True, token=token)

RepoUrl('https://huggingface.co/Youssef-El-SaYed/toxic-comment-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='Youssef-El-SaYed/toxic-comment-classifier')

In [13]:
api = HfApi()
api.upload_folder(
    folder_path=extract_dir,   
    repo_id=repo_id,
    repo_type="model",
    commit_message="Upload fine-tuned toxic comment classifier model",
    token=token
)

print(f"✅ Model uploaded: https://huggingface.co/{repo_id}")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...acts/checkpoint-15958/rng_state.pth:  78%|#######7  | 11.0kB / 14.2kB            

  ...acts/checkpoint-15500/rng_state.pth:  78%|#######7  | 11.0kB / 14.2kB            

  ...facts/checkpoint-15958/optimizer.pt:   0%|          |  643kB /  536MB            

...

  ...facts/checkpoint-15500/optimizer.pt:   0%|          |  644kB /  536MB            

  ...facts/checkpoint-15500/scheduler.pt:  10%|#         |   110B / 1.06kB            

  .../checkpoint-15500/training_args.bin:  10%|#         |   546B / 5.24kB            

  ...facts/checkpoint-15958/scheduler.pt:  10%|#         |   110B / 1.06kB            

✅ Model uploaded: https://huggingface.co/Youssef-El-SaYed/toxic-comment-classifier


In [15]:
model_id = "Youssef-El-SaYed/toxic-comment-classifier"

# Load from Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

# Build pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Test examples
print(pipe("I hate you so much!"))
print(pipe("What a wonderful day, thank you!"))

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.9894742369651794}]
[{'label': 'LABEL_0', 'score': 0.9999063014984131}]


In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
 
model_id = "Youssef-El-SaYed/toxic-comment-classifier"

# Define mapping
id2label = {0: "Non-Toxic", 1: "Toxic"}
label2id = {"Non-Toxic": 0, "Toxic": 1}

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    id2label=id2label,
    label2id=label2id
)

nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

print(nlp("You are so stupid and annoying!"))  
print(nlp("I really like your work, keep it up!"))


Device set to use cpu


[{'label': 'Toxic', 'score': 0.9931373596191406}]
[{'label': 'Non-Toxic', 'score': 0.9997437596321106}]
